# Washington Public Safety Preliminary Analysis
## 8 September 2020

As part of the Fall preparation sprint, we spent the last ten days analyzing what it will take to beat the infection decisively this fall. There are three fundamental ideas:
1. Backstop. Have a backstop that protects hospitals in the unlikely event, they outstrip their ability to procure the right resources. They are our last line of defense against the infection.
2. Purchasing Cooperative. For smaller groups that have funds, but who can benefit from the aggregation of supply. This will protect an important group that is upstream of hospitals.
3. Protecting the most vulnerable. The infection starts at this front line. We need to defend the most vulnerable against the infection. Not only does this reduce the disease burden, it also increases employment and takes the pressure off our relief efforts. Winning upstream, winning at the front lines, that is the key to victory.

Questions and edits to rich@restart.us and lucas@restart.us

In [1]:
import sys
import pandas as pd
import ipywidgets as widgets
from restart import RestartModel
from restart.util import set_config, to_df, to_sheet, display_population, format_population, format_cells

# Public Safety Groupings (for Des)
These include EMTs, firefighters, and other public safety officers. A detailed breakdown of SOC codes can be seen below.

In [8]:
config = set_config('../restart')
restart = RestartModel(
    config_dir='../config/wa_groups',
    data_dir='../../data/ingestion',
    population='oes',
    state='Washington',
    subpop='wa_groupings'
)

model = restart.model
model.inventory.set_average_orders_per_period(model.demand.demand_by_popsum1_total_rp1n_tc)

slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard(backstop):
    set_stock(backstop)
    
def display_stock(df):
    df = df.round()
    index_name = "Population"
    headers = ['EMTs', 'Firefighters', 'Other Public Safety']
    df.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df)
    format_cells(sheet)
    sheet.row_headers = False
    display(sheet)
    
def set_stock(backstop):
    backstop = [backstop]
    model.inventory.order(model.inventory.inv_by_popsum1_total_rp1n_tc)
    model.inventory.set_min_in_periods(backstop)
    display_stock(model.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
wa_burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))
pop = format_population(to_sheet(model.population.population_pP_tr.df))
    
out = widgets.interactive_output(dashboard, {'backstop': slider})

widgets.VBox([slider, out])

ValueError: Shape of passed values is (727, 2), indices imply (727, 3)

In [7]:
sum_df = restart.model.population.pop_to_popsum1_per_unit_map_pp1_us.df
display_population(to_sheet(sum_df))

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format='0,000', read_only=True, row_end=6, row_start=0…

# Washington Grouping Analysis

# Group 1: Backstop to Large Providers
These are the large hospitals and other facilities that take care of our sickest patients. With the fall, we will face the difficult challenge of both the flu season and the recurrence of COVID-19 as the climate worsens and people move indoors.

This group will provide most of their own PPE, disinfection and other resources with the state acting as a backstop in extreme emergencies. Given the exponential nature of infection, we need this backstop because even the best predictions have a large variance. For example, if the disease doubles every week, then even a two week error in estimate will increase PPE requirements in COVID wards by 4x.

# Group 2: Aggregate Demand of Smaller Providers

This second group consists of several different populations

- Smaller health care providers such as hospitals with 299 beds or less
- long-term care providers and nursing homes
- behavioral health facilities
- dentists
- morticians
- Federally Qualified Health Centers [FQHC](https://www.hrsa.gov/opa/eligibility-and-registration/health-centers/fqhc/index.html) community health providers
- public health organizations,
- tribal clinics
- independent physician practices
- First responders including EMOs, police, fire

We use two different methods to estimate these populations. More detailed surveys and census methods are also possible, so consider these methods as ways to get a broad measure of the scope of the problem. 

## Employee Classification (SOC) Analysis

SOC codes starting with "29-", "31-", and "33-" refer to, respectively, healthcare occupations, healthcare support occupations, and protection services. We estimate of the percentage of healthcare workers or healthcare-support workers fall into the Group 2 category, and simply scale all the numbers by that amount. It seems like a safe assumption that all the protection services would fall into this group. 

This analysis provides the stockpile that you would need to 100% cover the group for 30 days. A key policy decision is the size of the back stop needed. If you want to cover 50% of the demand for 30 days, then the figures would be half that.


In [8]:
import numpy as np
import ipysheet
import ipywidgets as widgets
from ipywidgets import Layout

config = set_config('../restart')
restart = RestartModel(configdir='../restart', population='oes', state='Washington', subpop='wa_tier2_opt1')
model = restart.model
model.inventory.set_average_orders_per_period(model.demand.demand_by_popsum1_total_rp1n_tc)

slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard(backstop):
    set_stock(backstop)
    
def display_stock(df):
    df = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df)
    format_cells(sheet)
    sheet.row_headers = False
    display(sheet)
    
def set_stock(backstop):
    backstop = [backstop]
    model.inventory.order(model.inventory.inv_by_popsum1_total_rp1n_tc)
    model.inventory.set_min_in_periods(backstop)
    display_stock(model.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
wa_burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))
pop = format_population(to_sheet(model.population.population_pP_tr.df))
    
out = widgets.interactive_output(dashboard, {'backstop': slider})

widgets.VBox([slider, out])

Burn rate assumptions and a detailed population breakdown

In [9]:
wa_burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))
pop = format_population(to_sheet(model.population.population_pP_tr.df))

widgets.VBox([wa_burn_sheet, pop])

Mapping of population to essential vs. non-essential workers

In [10]:
essential_arr = restart.model.population.pop_to_popsum1_per_unit_map_pp1_us.array
population_labels = list(restart.model.population.pop_to_popsum1_per_unit_map_pp1_us.df.index)


essential_list = []

for row in essential_arr:
    if row[0] == 1:
        essential_list.append('Y')
    else:
        essential_list.append('N')

essential_df = pd.DataFrame(essential_list)
essential_df.insert(loc=0, column='Population', value=population_labels)
essential_df.columns = ['Population', 'Essential Worker']
essential_sheet = to_sheet(essential_df)
essential_sheet.row_headers = False

essential_sheet

Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format=None, row_end=86, row_start=0, squeeze_row=Fals…

### Note on specific SOC codes used in Analysis

The main assumption here is that we are going to include certain groups as part of the coding for Group 2 this would include, a subset of occupations in these groups. Note that there is some overlap here because this is a list of all occupations and some percentage of these work in Group 1. These can of course all be modified, but this is the basis for the "quick" pull done here:

| OCC Category  | SOC Code |
| :- | -: |
| OCC Category 29 | SOC Code |
| Dental Hygienists | 29-1292 |   
| EMTs and Paramedics | 29-2040 |
| Family Medicine Physicians | 29-1215 | 
| Respiratory Therapists | 29-1126 |
| Psychiatrists | 29-1223 |
| Audiologists | 29-1181 |
| Pediatricians | 29-1221 |
| Psychiatric Technicians | 29-2052 |

| OCC Category 31 | SOC Code |
| :- | -: |
| Home Health and Personal Care Aides | 31-1120 |
| Nursing Assistants | 31-1131   |
| Morticians, Undertakers, and Funeral Arrangers |  39-4031 |
| Orderlies | 31-1132 |
| Psychiatric Aides | 31-1133  |
| Dental Assistants | 31-9091 |

| OCC Category 33 and 39 | SOC Code |
| :- | -: |
| Firefighters |  33-2011 |
| Correctional Officers and Jailers |  33-3012 |
| Detectives and Criminal Investigators |  33-3021  |
| Transportation Security Screeners | 33-9093 |
| Parking Enforcement Workers | 33-3041   |
| Police and Sheriff’s Patrol officers | 33-3051 |
| Embalmers |  39-4011 |


Pulling from these pre-defined codes, this is the resulting analysis:

In [11]:
config = set_config('../restart')
restart = RestartModel(configdir='../restart', population='oes', state='Washington', subpop='wa_tier2_opt2')
model = restart.model
model.inventory.set_average_orders_per_period(model.demand.demand_by_popsum1_total_rp1n_tc)

slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard(backstop):
    set_stock(backstop)
    
def display_stock(df):
    df = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df)
    format_cells(sheet)
    sheet.row_headers = False
    display(sheet)
    
def set_stock(backstop):
    backstop = [backstop]
    model.inventory.order(model.inventory.inv_by_popsum1_total_rp1n_tc)
    model.inventory.set_min_in_periods(backstop)
    display_stock(model.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
wa_burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))
pop = format_population(to_sheet(model.population.population_pP_tr.df))
    
out = widgets.interactive_output(dashboard, {'backstop': slider})

widgets.VBox([slider, out])

Burn rates and population breakdown:

In [12]:
widgets.VBox([wa_burn_sheet, pop])